In [1]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.models import SLDAResNetModel
from avalanche.training import StreamingLDA
from avalanche.logging import InteractiveLogger
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [3]:
splitcifar = SplitCIFAR10(n_experiences=5, seed=seed, dataset_root='../data')

train_set = splitcifar.train_stream
test_set = splitcifar.test_stream

print(splitcifar.classes_order)

In [4]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(run_name="SLDA-CIFAR10")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_class = 10

model = SLDAResNetModel(imagenet_pretrained=False, device=device)

# Prepare for training & testing
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [6]:
epoch = 10
train_batch = 128
eval_batch = 64

strategies = StreamingLDA(model, criterion, input_size=32, num_classes=num_class, train_epochs=epoch, train_mb_size=train_batch, eval_mb_size=eval_batch, device=device, evaluator=eval_plugin)

In [7]:
for i, exp in enumerate(train_set):
    eval_exps = test_set[i]
    # eval_exps = [e for e in test_set][i]
    strategies.train(exp)
    strategies.eval(eval_exps)

In [8]:
epoch = 10
train_batch = 128
eval_batch = 64

strategies = StreamingLDA(model, criterion, input_size=512, num_classes=num_class, train_epochs=epoch, train_mb_size=train_batch, eval_mb_size=eval_batch, device=device, evaluator=eval_plugin)

In [9]:
for i, exp in enumerate(train_set):
    eval_exps = test_set[i]
    # eval_exps = [e for e in test_set][i]
    strategies.train(exp)
    strategies.eval(eval_exps)

In [10]:
for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)

In [11]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.models import SLDAResNetModel
from avalanche.training import StreamingLDA
from avalanche.logging import InteractiveLogger
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

In [12]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [13]:
splitcifar = SplitCIFAR10(n_experiences=5, seed=seed, dataset_root='../data')

train_set = splitcifar.train_stream
test_set = splitcifar.test_stream

print(splitcifar.classes_order)

In [14]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(run_name="SLDA-CIFAR10")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_class = 10
incremental = 5
fixed_class_order = [4, 1, 7, 5, 3, 9, 0, 8, 6, 2]

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=True,
                        seed=seed,
                        shuffle=False,
                        fixed_class_order=fixed_class_order,
                        )

model = SLDAResNetModel(imagenet_pretrained=False, device=device)

# Prepare for training & testing
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [16]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.models import SLDAResNetModel
from avalanche.training import StreamingLDA
from avalanche.logging import InteractiveLogger
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

In [17]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [18]:
train_set = CIFAR10('../data', train=True, download=True)
test_set = CIFAR10('../data', train=False, download=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

train_set = AvalancheDataset(train_set, transform=train_transform)
test_set = AvalancheDataset(test_set, transform=eval_transform)

In [19]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(run_name="SLDA-CIFAR10")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_class = 10
incremental = 5
fixed_class_order = [4, 1, 7, 5, 3, 9, 0, 8, 6, 2]

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=True,
                        seed=seed,
                        shuffle=False,
                        fixed_class_order=fixed_class_order,
                        )

model = SLDAResNetModel(imagenet_pretrained=False, device=device)

# Prepare for training & testing
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [21]:
epoch = 10
train_batch = 128
eval_batch = 64

strategies = StreamingLDA(model, criterion, input_size=512, num_classes=num_class, train_epochs=epoch, train_mb_size=train_batch, eval_mb_size=eval_batch, device=device, evaluator=eval_plugin)

In [22]:
for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)

In [23]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.models import SLDAResNetModel
from avalanche.training import StreamingLDA
from avalanche.logging import InteractiveLogger
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

In [24]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [25]:
train_set = CIFAR10('../data', train=True, download=True)
test_set = CIFAR10('../data', train=False, download=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

train_set = AvalancheDataset(train_set, transform=train_transform)
test_set = AvalancheDataset(test_set, transform=eval_transform)

In [26]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(run_name="SLDA-CIFAR10")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])